# Scraping Jumia Ecommerce Site: DSE HACKATHON

## Import libraries

In [76]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
import os
import time
import re

import scripts.gemini_api_call as gemini_call
import importlib

In [77]:
# # start_time = time.time()
# importlib.reload(gemini_call)
# product_cat = gemini_call.predict_product_category("Infinix Note 30 4G, 6.66"", 256GB + 8GB RAM (Dual SIM), 5000mAh, Obsidian Black")
# # end_time = time.time()
# # elapsed_time = end_time - start_time
# # elapsed_time

In [79]:
# product_cat

## Define function to fetch website html data

In [4]:
def fetch_html_data(web_address):
    try:
        # print(f"\nFetching data from {web_address}...")
        res = req.get(web_address)
        return res
    except req.exceptions.RequestException as e:
        print('Stopped:', e)
    except TypeError as e:
        print('Stopped:', e)

# Define function to convert html data to BeautifulSoup object 

In [5]:
def convert_web_data_to_beautiful_soup_obj(web_data):
    try:
        # print("Creating BeautifulSoup object...")
        soup_obj = BeautifulSoup(web_data.text, "html.parser")
        # print("Success! Object created!")
        return soup_obj
    except Exception as e:
        print("Stopped:", e)

### Create array of products on each page, add each to product dictionary & append to array iteratively

In [6]:
all_products_list = []

def append_one_product_details_dictionary_to_list(pg):
    # print(f"Appending page {pg} products' details to array")
    
    page_products_details_soup = soup.find_all("article", class_="prd _fb col c-prd")
    
    for detail in page_products_details_soup:
        details_dict = {"name": detail.find("h3", class_="name").text.strip(),
                        "new_price": detail.find("div", class_="prc").text.strip(),
                        "old_price": detail.find("div", class_="old").text.strip() if detail.find("div", class_="old") else None,
                        "discount": detail.find("div", class_="bdg _dsct _sm").text.strip() if detail.find("div", class_="bdg _dsct _sm") else None,
                        "rating": detail.find("div", class_="stars _s").text.strip() if detail.find("div", class_="stars _s") else None,
                        "votes": detail.find("div", class_="rev").text.strip() if detail.find("div", class_="rev") else None}
        all_products_list.append(details_dict)


# len(product_details_clean)

### Fetch webdata, convert to BeautifulSoup Object, add product details dictionary to list

In [7]:
# Declare the products’ url & the total webpage count
other_pages_url = "https://www.jumia.co.ke/all-products/?page="
webpage_num_total = 50

# If scraped products csv doesn't exist, run the scraper function below & save products to csv ELSE continue & read csv
if not os.path.exists('../data/raw/all_products_list_raw.csv'):
    for page in range(1, webpage_num_total+1, 1):
        other_pages_url = "https://www.jumia.co.ke/all-products/?page="
        page = str(page)
        other_pages_url = other_pages_url + page
        response = fetch_html_data(other_pages_url)
        # time.sleep(2.5)
        soup = convert_web_data_to_beautiful_soup_obj(response)
        append_one_product_details_dictionary_to_list(page)
    
    # Convert scraped list to dataframe
    df = pd.DataFrame(all_products_list)
    # Save scraped dataframe to csv to prevent scraping each time I run the notebook
    df.to_csv('../data/raw/all_products_list_raw.csv')

### Check product count

In [8]:
# Read csv and specify index column to prevent creation of another redundant index
products_df = pd.read_csv('../data/raw/all_products_list_raw.csv', index_col=0)  # Read products list from csv

### Inspect products list dataframe

In [9]:
products_df.head()

,name,new_price,old_price,discount,rating,votes
0,NIVEA Radiant & Beauty Advanced Care Lotion Fo...,KSh 854,"KSh 1,460",42%,4.7 out of 5,4.7 out of 5(636)
1,"Vitron HTC4388FS - 43"" Smart Android Frameles...","KSh 21,506","KSh 28,599",25%,4.2 out of 5,4.2 out of 5(335)
2,NIVEA Perfect & Radiant Even Tone Day And Nigh...,KSh 899,"KSh 1,560",42%,4.5 out of 5,4.5 out of 5(2238)
3,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,KSh 693,"KSh 1,399",50%,4.1 out of 5,4.1 out of 5(1661)
4,Nunix R5 Hot & Normal Free Standing Water Disp...,"KSh 4,180",NaN,NaN,4.1 out of 5,4.1 out of 5(84)


In [10]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       2000 non-null   object
 1   new_price  2000 non-null   object
 2   old_price  1907 non-null   object
 3   discount   1907 non-null   object
 4   rating     109 non-null    object
 5   votes      109 non-null    object
dtypes: object(6)
memory usage: 109.4+ KB


In [11]:
products_df.shape

(2000, 6)

# Data Cleaning Section

### Clean new_price column

In [12]:
products_df['new_price'][1]

'KSh 21,506'

In [13]:
# Remove currency name 'Ksh' 
products_df['new_price'] = products_df['new_price'].apply(lambda x: x.split(' ')[1] if x is not None and isinstance(x,str) else x) 

In [14]:
print(type(products_df['new_price'][1]))
print(products_df['new_price'].head())

<class 'str'>
0       854
1    21,506
2       899
3       693
4     4,180
Name: new_price, dtype: object


In [15]:
# Remove comma from thousands price values
products_df['new_price'] = products_df['new_price'].apply(lambda x: x.replace(',','') if ',' in x and isinstance(x,str) else x)
products_df['new_price'].head()

0      854
1    21506
2      899
3      693
4     4180
Name: new_price, dtype: object

In [16]:
print(type(products_df['new_price'][1]))

<class 'str'>


In [17]:
# Convert new_price column to float
products_df['new_price'] = products_df['new_price'].apply(lambda x: float(x) if x is not None else x)

In [18]:
print(type(products_df['new_price'][1]))
print(products_df['new_price'].dtype)

<class 'numpy.float64'>
float64


In [19]:
products_df.head()

,name,new_price,old_price,discount,rating,votes
0,NIVEA Radiant & Beauty Advanced Care Lotion Fo...,854.0,"KSh 1,460",42%,4.7 out of 5,4.7 out of 5(636)
1,"Vitron HTC4388FS - 43"" Smart Android Frameles...",21506.0,"KSh 28,599",25%,4.2 out of 5,4.2 out of 5(335)
2,NIVEA Perfect & Radiant Even Tone Day And Nigh...,899.0,"KSh 1,560",42%,4.5 out of 5,4.5 out of 5(2238)
3,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,693.0,"KSh 1,399",50%,4.1 out of 5,4.1 out of 5(1661)
4,Nunix R5 Hot & Normal Free Standing Water Disp...,4180.0,NaN,NaN,4.1 out of 5,4.1 out of 5(84)


In [20]:
# Saving stage one (new_price) data cleaning file to csv
products_df.to_csv("../data/processed/1_all_products_new_price_cleaned.csv")

### Clean old_price column

In [21]:
# Read csv file cleaned in the previous stage (new_price column) & convert to dataframe
products_df = pd.read_csv("../data/processed/1_all_products_new_price_cleaned.csv", index_col=0)

# Use [:10] notation to print the first 10 values
products_df['old_price'].unique()[:10]

array(['KSh 1,460', 'KSh 28,599', 'KSh 1,560', 'KSh 1,399', nan,
       'KSh 1,750', 'KSh 24,999', 'KSh 4,099', 'KSh 19,999', 'KSh 1,299'],
      dtype=object)

In [22]:
# Remove currency name 'Ksh'
# type(products_df['old_price'][1])
products_df['old_price'] = products_df['old_price'].apply(lambda x: x.split(' ')[1] if x is not None and isinstance(x,str) else x)
products_df['old_price'].head()

0     1,460
1    28,599
2     1,560
3     1,399
4       NaN
Name: old_price, dtype: object

In [23]:
# Remove commas from values
products_df['old_price'] = products_df['old_price'].apply(lambda x: x.replace(',','') if x is not None and isinstance(x,str) else x)
products_df['old_price'].head(10)

0     1460
1    28599
2     1560
3     1399
4      NaN
5     1750
6    24999
7      NaN
8     4099
9    19999
Name: old_price, dtype: object

In [24]:
# Convert the string values into floats
products_df['old_price'] = products_df['old_price'].apply(lambda x: float(x) if x is not None else x)
products_df['old_price'].head(10)

0     1460.0
1    28599.0
2     1560.0
3     1399.0
4        NaN
5     1750.0
6    24999.0
7        NaN
8     4099.0
9    19999.0
Name: old_price, dtype: float64

In [25]:
products_df['old_price']

0        1460.0
1       28599.0
2        1560.0
3        1399.0
4           NaN
         ...   
1995     6638.0
1996     1284.0
1997     5602.0
1998     1427.0
1999     2867.0
Name: old_price, Length: 2000, dtype: float64

In [26]:
# Index 18 from above was null. So checking its type: 
print(type(products_df['old_price'][18]))
print(products_df['old_price'].dtype)

<class 'numpy.float64'>
float64


In [27]:
products_df.head()

,name,new_price,old_price,discount,rating,votes
0,NIVEA Radiant & Beauty Advanced Care Lotion Fo...,854.0,1460.0,42%,4.7 out of 5,4.7 out of 5(636)
1,"Vitron HTC4388FS - 43"" Smart Android Frameles...",21506.0,28599.0,25%,4.2 out of 5,4.2 out of 5(335)
2,NIVEA Perfect & Radiant Even Tone Day And Nigh...,899.0,1560.0,42%,4.5 out of 5,4.5 out of 5(2238)
3,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,693.0,1399.0,50%,4.1 out of 5,4.1 out of 5(1661)
4,Nunix R5 Hot & Normal Free Standing Water Disp...,4180.0,NaN,NaN,4.1 out of 5,4.1 out of 5(84)


In [28]:
# Saving stage two data (old_price column) cleaning dataframe to csv
products_df.to_csv("../data/processed/2_all_products_old_price_cleaned.csv")

### Cleaning 'discount' column

In [29]:
# Read csv file from stage 2 (old_price column cleanup) to get dataframe
products_df = pd.read_csv("../data/processed/2_all_products_old_price_cleaned.csv", index_col=0)

# Use [:10] notation to print the first 10 values
products_df['discount'].unique()[:10]

array(['42%', '25%', '50%', nan, '20%', '24%', '29%', '12%', '35%', '33%'],
      dtype=object)

In [30]:
# Remove the percent sign
products_df['discount'] = products_df['discount'].apply(lambda x: x.replace('%','') if x is not None and isinstance(x,str) else x)
products_df['discount'].unique()

array(['42', '25', '50', nan, '20', '24', '29', '12', '35', '33', '28',
       '40', '37', '27', '31', '47', '41', '44', '13', '30', '36', '8',
       '43', '49', '21', '46', '18', '19', '26', '45', '32', '22', '4',
       '48', '15', '38', '11', '34', '16', '23', '17', '1', '39', '5',
       '14', '10', '2', '3'], dtype=object)

In [31]:
# Convert values into float & divide by 100 to represent the percentages as decimal values
products_df['discount'] = products_df['discount'].apply(lambda x: float(x)/100 if x is not None else x)
products_df['discount'].unique()  # A numbers now floats and None replaced with NaN

array([0.42, 0.25, 0.5 ,  nan, 0.2 , 0.24, 0.29, 0.12, 0.35, 0.33, 0.28,
       0.4 , 0.37, 0.27, 0.31, 0.47, 0.41, 0.44, 0.13, 0.3 , 0.36, 0.08,
       0.43, 0.49, 0.21, 0.46, 0.18, 0.19, 0.26, 0.45, 0.32, 0.22, 0.04,
       0.48, 0.15, 0.38, 0.11, 0.34, 0.16, 0.23, 0.17, 0.01, 0.39, 0.05,
       0.14, 0.1 , 0.02, 0.03])

In [32]:
# Check head once more
products_df.head()

,name,new_price,old_price,discount,rating,votes
0,NIVEA Radiant & Beauty Advanced Care Lotion Fo...,854.0,1460.0,0.42,4.7 out of 5,4.7 out of 5(636)
1,"Vitron HTC4388FS - 43"" Smart Android Frameles...",21506.0,28599.0,0.25,4.2 out of 5,4.2 out of 5(335)
2,NIVEA Perfect & Radiant Even Tone Day And Nigh...,899.0,1560.0,0.42,4.5 out of 5,4.5 out of 5(2238)
3,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,693.0,1399.0,0.50,4.1 out of 5,4.1 out of 5(1661)
4,Nunix R5 Hot & Normal Free Standing Water Disp...,4180.0,NaN,NaN,4.1 out of 5,4.1 out of 5(84)


In [33]:
# Saving stage three data cleaning file to csv
products_df.to_csv("../data/processed/3_all_products_discount_cleaned.csv")

### Cleaning the rating column

In [34]:
# Read file cleaned in prior stage (discount column) to get dataframe
products_df = pd.read_csv("../data/processed/3_all_products_discount_cleaned.csv", index_col=0)

# Check for unique values first
products_df['rating'].unique()[:10]

array(['4.7 out of 5', '4.2 out of 5', '4.5 out of 5', '4.1 out of 5',
       '4.3 out of 5', '3.8 out of 5', '3.9 out of 5', '4 out of 5',
       '4.4 out of 5', '4.6 out of 5'], dtype=object)

In [35]:
# Split rating string values into 4 elements and select the first element unless its None(left as is)
products_df['rating'] = products_df['rating'].apply(lambda x: x.split(' ')[0] if x is not None and isinstance(x,str) else x)
products_df['rating'].unique()

array(['4.7', '4.2', '4.5', '4.1', '4.3', '3.8', '3.9', '4', '4.4', '4.6',
       '3.6', '3.7', nan, '4.8', '3.4', '5', '1', '1.7', '2'],
      dtype=object)

In [36]:
# Convert the values into float
products_df['rating'] = products_df['rating'].apply(lambda x: float(x) if x is not None else x)
print(products_df['rating'].dtype)

float64


In [37]:
products_df.head(20)

,name,new_price,old_price,discount,rating,votes
0,NIVEA Radiant & Beauty Advanced Care Lotion Fo...,854.0,1460.0,0.42,4.7,4.7 out of 5(636)
1,"Vitron HTC4388FS - 43"" Smart Android Frameles...",21506.0,28599.0,0.25,4.2,4.2 out of 5(335)
2,NIVEA Perfect & Radiant Even Tone Day And Nigh...,899.0,1560.0,0.42,4.5,4.5 out of 5(2238)
3,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,693.0,1399.0,0.50,4.1,4.1 out of 5(1661)
4,Nunix R5 Hot & Normal Free Standing Water Disp...,4180.0,NaN,NaN,4.1,4.1 out of 5(84)
5,Garnier Anti-Blemish Charcoal Serum With AHA +...,1400.0,1750.0,0.20,4.3,4.3 out of 5(1455)
6,"Tecno Spark 10 , 6.6"" FHD,128GB ROM+8GB RAM, 5...",18999.0,24999.0,0.24,3.8,3.8 out of 5(17)
7,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,628.0,NaN,NaN,4.7,4.7 out of 5(6)
8,AILYONS ELP2404K 2.1CH SubWoofer Home Theatrer...,2906.0,4099.0,0.29,3.9,3.9 out of 5(98)
9,"Nokia C32, 64GB + 4GB (Upto 7GB RAM) 5000MAh, ...",17599.0,19999.0,0.12,4.3,4.3 out of 5(95)


In [38]:
# Saving stage four data cleaning file to csv
products_df.to_csv("../data/processed/4_all_products_rating_cleaned.csv")

### Cleaning the 'votes' column

In [39]:
# Read csv file saved in prior cleaning stage (rating column) to obtain dataframe
products_df = pd.read_csv("../data/processed/4_all_products_rating_cleaned.csv", index_col=0)

# Explore unique values
products_df['votes'].unique()[:10]

array(['4.7 out of 5(636)', '4.2 out of 5(335)', '4.5 out of 5(2238)',
       '4.1 out of 5(1661)', '4.1 out of 5(84)', '4.3 out of 5(1455)',
       '3.8 out of 5(17)', '4.7 out of 5(6)', '3.9 out of 5(98)',
       '4.3 out of 5(95)'], dtype=object)

In [40]:
# Extract the votes count with string manipulation methods & convert values to floats
products_df['votes'] = products_df['votes'].apply(lambda x: float(x.split('(')[1].split(')')[0]) if x is not None and isinstance(x,str) else x)
products_df['votes'].unique()[:15]

array([ 636.,  335., 2238., 1661.,   84., 1455.,   17.,    6.,   98.,
         95.,  747.,  926.,  215.,   77.,  367.])

In [41]:
products_df.head(20)

,name,new_price,old_price,discount,rating,votes
0,NIVEA Radiant & Beauty Advanced Care Lotion Fo...,854.0,1460.0,0.42,4.7,636.0
1,"Vitron HTC4388FS - 43"" Smart Android Frameles...",21506.0,28599.0,0.25,4.2,335.0
2,NIVEA Perfect & Radiant Even Tone Day And Nigh...,899.0,1560.0,0.42,4.5,2238.0
3,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,693.0,1399.0,0.50,4.1,1661.0
4,Nunix R5 Hot & Normal Free Standing Water Disp...,4180.0,NaN,NaN,4.1,84.0
5,Garnier Anti-Blemish Charcoal Serum With AHA +...,1400.0,1750.0,0.20,4.3,1455.0
6,"Tecno Spark 10 , 6.6"" FHD,128GB ROM+8GB RAM, 5...",18999.0,24999.0,0.24,3.8,17.0
7,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,628.0,NaN,NaN,4.7,6.0
8,AILYONS ELP2404K 2.1CH SubWoofer Home Theatrer...,2906.0,4099.0,0.29,3.9,98.0
9,"Nokia C32, 64GB + 4GB (Upto 7GB RAM) 5000MAh, ...",17599.0,19999.0,0.12,4.3,95.0


In [42]:
# Saving stage five data cleaning file to csv
products_df.to_csv("../data/processed/5_all_products_votes_cleaned.csv")

## Dealing with null values

In [43]:
# Read csv file from prior cleaning (votes column) to get dataframe
products_df = pd.read_csv("../data/processed/5_all_products_votes_cleaned.csv", index_col=0)

In [44]:
products_df.isnull().sum().idxmax()

'rating'

In [45]:
print(f"new_price nulls: {products_df.new_price.isnull().sum()}")
print(f"old_price nulls: {products_df.old_price.isnull().sum()}")
print(f"discount nulls: {products_df.discount.isnull().sum()}")
print(f"rating nulls: {products_df.rating.isnull().sum()}")
print(f"votes nulls: {products_df.votes.isnull().sum()}")

new_price nulls: 0
old_price nulls: 93
discount nulls: 93
rating nulls: 1891
votes nulls: 1891


### Fill null values in discount column 

In [46]:
# Use discount column average to fill nulls
products_df['discount'] = products_df['discount'].fillna(round(products_df['discount'].mean(), 2))
products_df.discount.isnull().sum()

0

### Fill null values in old_price based on the discount column average

In [47]:
def multiply_fill_old_price_column_nulls(row):
    new_price = row['new_price']
    discount_column_mean = round(products_df['discount'].mean(), 2)
    
    if not pd.isna(row['old_price']):
        return row['old_price']
    return new_price * discount_column_mean
        
products_df['old_price'] = products_df.apply(multiply_fill_old_price_column_nulls, axis=1)
products_df.old_price.isnull().sum()

0

### Fill rating nulls with column mean

In [48]:
products_df['rating'] = products_df['rating'].fillna(round(products_df['rating'].mean(), 2))
products_df.rating.isnull().sum()
# type(products_df['rating'][1])

0

### Fill votes nulls with column mean

In [49]:
products_df['votes'] = products_df['votes'].fillna(round(products_df['votes'].mean(), 2))
products_df.votes.isnull().sum()

0

## Rechecking nulls for all columns again

In [50]:
print(f"new_price nulls: {products_df.new_price.isnull().sum()}")
print(f"old_price nulls: {products_df.old_price.isnull().sum()}")
print(f"discount nulls: {products_df.discount.isnull().sum()}")
print(f"rating nulls: {products_df.rating.isnull().sum()}")
print(f"votes nulls: {products_df.votes.isnull().sum()}")

new_price nulls: 0
old_price nulls: 0
discount nulls: 0
rating nulls: 0
votes nulls: 0


In [51]:
products_df.head(10)

,name,new_price,old_price,discount,rating,votes
0,NIVEA Radiant & Beauty Advanced Care Lotion Fo...,854.0,1460.00,0.42,4.7,636.0
1,"Vitron HTC4388FS - 43"" Smart Android Frameles...",21506.0,28599.00,0.25,4.2,335.0
2,NIVEA Perfect & Radiant Even Tone Day And Nigh...,899.0,1560.00,0.42,4.5,2238.0
3,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,693.0,1399.00,0.50,4.1,1661.0
4,Nunix R5 Hot & Normal Free Standing Water Disp...,4180.0,1713.80,0.41,4.1,84.0
5,Garnier Anti-Blemish Charcoal Serum With AHA +...,1400.0,1750.00,0.20,4.3,1455.0
6,"Tecno Spark 10 , 6.6"" FHD,128GB ROM+8GB RAM, 5...",18999.0,24999.00,0.24,3.8,17.0
7,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,628.0,257.48,0.41,4.7,6.0
8,AILYONS ELP2404K 2.1CH SubWoofer Home Theatrer...,2906.0,4099.00,0.29,3.9,98.0
9,"Nokia C32, 64GB + 4GB (Upto 7GB RAM) 5000MAh, ...",17599.0,19999.00,0.12,4.3,95.0


In [52]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       2000 non-null   object 
 1   new_price  2000 non-null   float64
 2   old_price  2000 non-null   float64
 3   discount   2000 non-null   float64
 4   rating     2000 non-null   float64
 5   votes      2000 non-null   float64
dtypes: float64(5), object(1)
memory usage: 109.4+ KB


In [53]:
# Saving stage 6 data cleaning file to csv
products_df.to_csv("../data/final/6_all_products_all_columns_cleaned.csv")

In [90]:
experiment_df = pd.read_csv("../data/final/6_all_products_all_columns_cleaned.csv", index_col=0)
new_experiment_df = experiment_df.iloc[0:10]
new_experiment_df

,name,new_price,old_price,discount,rating,votes
0,NIVEA Radiant & Beauty Advanced Care Lotion Fo...,854.0,1460.00,0.42,4.7,636.0
1,"Vitron HTC4388FS - 43"" Smart Android Frameles...",21506.0,28599.00,0.25,4.2,335.0
2,NIVEA Perfect & Radiant Even Tone Day And Nigh...,899.0,1560.00,0.42,4.5,2238.0
3,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,693.0,1399.00,0.50,4.1,1661.0
4,Nunix R5 Hot & Normal Free Standing Water Disp...,4180.0,1713.80,0.41,4.1,84.0
5,Garnier Anti-Blemish Charcoal Serum With AHA +...,1400.0,1750.00,0.20,4.3,1455.0
6,"Tecno Spark 10 , 6.6"" FHD,128GB ROM+8GB RAM, 5...",18999.0,24999.00,0.24,3.8,17.0
7,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,628.0,257.48,0.41,4.7,6.0
8,AILYONS ELP2404K 2.1CH SubWoofer Home Theatrer...,2906.0,4099.00,0.29,3.9,98.0
9,"Nokia C32, 64GB + 4GB (Upto 7GB RAM) 5000MAh, ...",17599.0,19999.00,0.12,4.3,95.0


In [100]:
# Reload the gemini api call module, 'gemini_call', to refresh/reload it before use to prevent any unusual errors
importlib.reload(gemini_call)

# def fetch_category_name(product_name):
#     product_cat = gemini_call.predict_product_category(product_name)
#     return product_cat 

durations_list = []
for idx, row in new_experiment_df.iterrows():
    name_of_product = row["name"]
    
    # Timed the API roundtrip during testing to find the average time...
    # ... to check if it's possible to surpass the set rate limit of 60 requests per min (RPM)...
    # ... besides, while doing the average time testing, I put a 1.2 sec sleep timer...
    # ... as the last line function as total guard to ensure a max of 50 RPM
    
    start_time = time.time()
    predicted_category = gemini_call.predict_product_category(name_of_product)
    end_time = time.time()
    durations_list.append(end_time - start_time)
    
    print(f"{name_of_product} = {predicted_category}")
    new_experiment_df.at[idx, "product_category"] = predicted_category
    
    # Commented the sleep timer after find the average roundtrip api call time is: 2.2 seconds 
    # time.sleep(1.2)
    

new_experiment_df

NIVEA Radiant & Beauty Advanced Care Lotion For Women - 400ml (Pack Of 2) = Health & Beauty
Vitron HTC4388FS - 43" Smart  Android Frameless TV - Black = Electronics
NIVEA Perfect & Radiant Even Tone Day And Night Cream For Women - 50ml = Health & Beauty
AILYONS FK-0301 Stainless Steel 1.8L Electric Kettle-Black = Home & Office
Nunix R5 Hot & Normal Free Standing Water Dispenser - White = Home & Office
Garnier Anti-Blemish Charcoal Serum With AHA + BHA & Niacinamide For Acne Prone Skin = Health & Beauty
Tecno Spark 10 , 6.6" FHD,128GB ROM+8GB RAM, 5000mAh,Blue = Phones & Tablets
AILYONS FK-0301 Stainless Steel 1.8L Electric Kettle-Silver. = Home & Office
AILYONS ELP2404K 2.1CH SubWoofer Home Theatrer Multi Media Speaker System = Electronics
Nokia C32, 64GB + 4GB (Upto 7GB RAM) 5000MAh, Android 13, Black-Charcoal = Phones & Tablets


,name,new_price,old_price,discount,rating,votes,product_category
0,NIVEA Radiant & Beauty Advanced Care Lotion Fo...,854.0,1460.00,0.42,4.7,636.0,Health & Beauty
1,"Vitron HTC4388FS - 43"" Smart Android Frameles...",21506.0,28599.00,0.25,4.2,335.0,Electronics
2,NIVEA Perfect & Radiant Even Tone Day And Nigh...,899.0,1560.00,0.42,4.5,2238.0,Health & Beauty
3,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,693.0,1399.00,0.50,4.1,1661.0,Home & Office
4,Nunix R5 Hot & Normal Free Standing Water Disp...,4180.0,1713.80,0.41,4.1,84.0,Home & Office
5,Garnier Anti-Blemish Charcoal Serum With AHA +...,1400.0,1750.00,0.20,4.3,1455.0,Health & Beauty
6,"Tecno Spark 10 , 6.6"" FHD,128GB ROM+8GB RAM, 5...",18999.0,24999.00,0.24,3.8,17.0,Phones & Tablets
7,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,628.0,257.48,0.41,4.7,6.0,Home & Office
8,AILYONS ELP2404K 2.1CH SubWoofer Home Theatrer...,2906.0,4099.00,0.29,3.9,98.0,Electronics
9,"Nokia C32, 64GB + 4GB (Upto 7GB RAM) 5000MAh, ...",17599.0,19999.00,0.12,4.3,95.0,Phones & Tablets


In [99]:
average_duration = sum(durations_list) / len(durations_list)
average_duration

2.235899829864502

### Generate product categories in a new column using GEMINI API generative AI (based on product names as prompt)>

In [ ]:
# # Reload the module to refresh it since it has been modified since being imported
# importlib.reload(gemini_call)
# 
# # def remove_non_alphanumeric(text):
# #     return re.sub(r'[^a-zA-Z0-9,.!\s]', '', text)
# 
# def fetch_category_name(product_name):
#     # name_has_non_al_num = not all(char.isalnum() for char in product_name)
#     # 
#     # if name_has_non_al_num:
#     #     cleaned_product_name = remove_non_alphanumeric(product_name)
#     #     product_cat = gemini_call.predict_product_category(cleaned_product_name)
#     #     return product_cat
#     # else:   
#         product_cat = gemini_call.predict_product_category(product_name)
#         return product_cat 
# 
# 
# # requests_counter = 0
# for idx, row in products_df.iterrows():
#     name_of_product = row["name"]
#     start_time = time.time()
#     products_df["product_category"] = fetch_category_name(name_of_product)
#     end_time = time.time()
#     print(f"{idx}: {end_time - start_time}")
#     
#     


### Assign cleaned dataframe to a new name

In [ ]:
# Read csv of the final data into dataframe
# products_clean_df = pd.read_csv("../data/final/6_all_products_all_columns_cleaned.csv", index_col=0)
# products_clean_df.head()
products_df.to_csv("../data/final/7_product_category_column_added.csv")
products_clean_df = pd.read_csv("../data/final/7_product_category_column_added.csv", index_col=0)
products_clean_df.iloc[1783]

In [ ]:
prod_name = products_clean_df["new_price"][0]
prod_name

# DATA VISUALIZATION SECTION

### Summary stats of the data

In [ ]:
summary_stats = products_clean_df.describe()
summary_stats

### Deriving the correlation matrix

In [ ]:
# Returning correlation matrix for only the numerical columns
correlation_matrix = products_clean_df.corr(method='pearson', numeric_only=True)
# correlation_matrix.to_csv("results/datafiles/pearson_correlation.csv")
correlation_matrix

In [ ]:
plt.xticks(np.arange(len(products_clean_df.columns)), products_clean_df.columns, rotation=45)
plt.yticks(np.arange(len(products_clean_df.columns)), products_clean_df.columns, rotation=45)

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of the Numeric Columns (Jumia data)', y=1.02)

plt.savefig("../results/plots/1_correlation_matrix_of_Jumia_products_numeric_data.png", bbox_inches="tight",pad_inches=2)

### Visualize top-rated products with at least 200 votes

In [ ]:
# First filter for products with at least 200 votes (More votes makes a product rating more reliable)
products_with_atleast_200_votes_df = products_clean_df[products_clean_df['votes'] >= 200]
products_with_atleast_200_votes_df

In [ ]:
# Check counts of products with rating at least 4.5
top_rated_products_df = products_with_atleast_200_votes_df[products_with_atleast_200_votes_df['rating'] >= 4.5]
top_rated_products_df

In [ ]:
# Selecting only votes & rating column before plotting
new_top_rated_products_df = top_rated_products_df[['name','rating', 'votes']].reset_index(drop=True)
new_top_rated_products_df

In [ ]:
# Creating a horizontal bar plot
plt.figure(figsize=(18,10))

plt.barh(new_top_rated_products_df['name'], new_top_rated_products_df['rating'])

plt.title("Top-rated products with at least 4.5 stars and 200 Votes")
plt.ylabel("Product", fontsize=7)
plt.xlabel("Rating")

# Save the plot to file
plt.savefig("../results/plots/2_products_4.5_stars_and_more_than_200_votes.png",bbox_inches='tight',pad_inches=2)

### Showing product price distribution: Histogram with Kernel density estimate (kde) curve

In [ ]:
plt.figure(figsize=(20, 5))
sns.histplot(products_clean_df['new_price'], kde=True)
plt.xlabel('Product Price')
plt.xticks(np.arange(0, max(products_clean_df['new_price'])+1, 4000),rotation=90)
plt.ylabel('Frequency')
plt.title('Distribution of Product Prices')

# Save the plot to file
plt.savefig("../results/plots/3_product_prices_histogram_with_kernel_estimate_curve.png",bbox_inches='tight',pad_inches=1)

### Visualizing discount distribution

In [ ]:
sns.histplot(products_clean_df['discount'])
plt.xlabel('Discount')
# plt.xticks(np.arange(0,max(products_clean_df['discount'])+1,0.5),rotation=90)
plt.xticks(rotation=90)
plt.ylabel('Frequency')
plt.title('Distribution of Discounts')

# Save the plot to file
plt.savefig("../results/plots/4_product_discount_histogram.png",bbox_inches='tight',pad_inches=1)

### Visualizing rating distribution

In [ ]:
sns.histplot(data=products_clean_df, x='rating')
plt.title('Distribution of Ratings')
plt.xticks(np.arange(1.0, max(products_clean_df['rating'])+1, 0.1),rotation=90,fontsize=8)
plt.ylabel("Frequency")

# Save plot to file
plt.savefig("../results/plots/5_ratings_distribution_histogram.png", bbox_inches="tight", pad_inches=1)

### Product Name Analysis

In [ ]:
words = ' '.join(products_clean_df['name']).split()
all_words = Counter(words)
# len(all_words)  # 4580

# Returning an array of tuples with all_words & their counts, then return first 10
all_words_counts = all_words.most_common()
all_words_counts[:10]

In [ ]:
# Return only the alphabetical elements & their counts, & return the first 10
only_alpha_words_list = [(word, count) for word, count in all_words_counts if word.isalpha()]
only_alpha_words_list[:10]

In [ ]:
# Select the top 30 words
first_30_words = only_alpha_words_list[:31]
first_30_words

# Future: Use library to remove conjuctions such as 'and' from this list

In [ ]:
# Plot a chart with the first 30 words
plt.bar([d[0] for d in first_30_words], [d[1] for d in first_30_words])
plt.xlabel("Product names")
plt.xticks(rotation=90)
plt.ylabel("Counts")
plt.title("Bar chart of Product Names & Values")

# Save plot to file
plt.savefig("../results/plots/6_barchart_top_30_product_names_counts.png", bbox_inches="tight", pad_inches=1)

### Creating pairplot showing relationship between various numerical columns

In [ ]:
sns.pairplot(products_clean_df)
plt.suptitle('Pair Plot of Numerical Features', y=1.02)

# Save to file
plt.savefig("../results/plots/7_pairplot_rltnships_between_all_numerical_columns.png", bbox_inches="tight", pad_inches=1)